<a href="https://colab.research.google.com/github/couch-taco/Advent-of-Code/blob/main/Day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Packages

In [ ]:
!pip install aocd

In [57]:
from aocd import get_data, submit
import pandas as pd
import numpy as np
import re

Load Data

In [13]:
puzzel_year = 2023
puzzel_day = 2
cookies = ''
raw = get_data(day = puzzel_day, year = puzzel_year, session= cookies)

In [ ]:
test_data = 'Game 1: 4 red, 3 blue; 6 blue, 16 green; 9 blue, 13 green, 1 red; 10 green, 4 red, 6 blue'
#Game 2: 2 green, 3 blue; 11 red; 2 green, 5 red, 1 blue Game 3: 19 green, 4 blue, 13 red; 1 green, 1 blue, 1 red; 17 red, 18 green'

In [22]:
data = raw

Clean Data

In [33]:
#raw_data = raw_data[1:]
#df = pd.DataFrame(data = test_data.split('Game'))
df = pd.DataFrame(data = data.split('Game'))
df.columns = ['games']
df[['gameID', 'results']] = df['games'].str.split(':',expand=True)
df = pd.concat([df, df['results'].str.split(';', expand=True)], axis=1)
df2 = df.drop(columns = ['games', 'results'])

In [35]:
df2

,gameID,0,1,2,3,4,5
0,,None,None,None,None,None,None
1,1,"4 red, 3 blue","6 blue, 16 green","9 blue, 13 green, 1 red","10 green, 4 red, 6 blue\n",None,None
2,2,"2 green, 3 blue",11 red,"2 green, 5 red, 1 blue\n",None,None,None
3,3,"19 green, 4 blue, 13 red","1 green, 1 blue, 1 red","17 red, 18 green\n",None,None,None
4,4,"4 green, 8 blue, 20 red","19 red, 3 green, 14 blue","15 red, 4 green, 1 blue","18 blue, 14 red","19 red, 10 blue","3 green, 11 blue, 15 red\n"
...,...,...,...,...,...,...,...
96,96,"1 red, 7 blue, 2 green",5 green,"3 red, 5 green, 11 blue\n",None,None,None
97,97,"8 green, 6 red","1 blue, 6 red, 10 green","1 blue, 6 red\n",None,None,None
98,98,"2 green, 8 red, 1 blue","9 green, 2 blue, 7 red","1 blue, 2 red, 11 green","8 red, 10 green, 2 blue\n",None,None
99,99,"3 blue, 2 red","1 blue, 3 green, 3 red","1 red, 3 green","2 green, 2 red, 2 blue\n",None,None


In [41]:
df_melt = pd.melt(df2, id_vars= ['gameID'], value_name= 'results')
df_melt = df_melt.dropna(subset= ['results'])

In [54]:
df_melt['red'] = df_melt['results'].str.extract(r"(\d+)(?=\s*red)")
df_melt['red'] = pd.to_numeric(df_melt['red'])
df_melt['blue'] = df_melt['results'].str.extract(r"(\d+)(?=\s*blue)")
df_melt['blue'] = pd.to_numeric(df_melt['blue'])
df_melt['green'] = df_melt['results'].str.extract(r"(\d+)(?=\s*green)")
df_melt['green'] = pd.to_numeric(df_melt['green'])
df_melt

,gameID,variable,results,red,blue,green
1,1,0,"4 red, 3 blue",4.0,3.0,NaN
2,2,0,"2 green, 3 blue",NaN,3.0,2.0
3,3,0,"19 green, 4 blue, 13 red",13.0,4.0,19.0
4,4,0,"4 green, 8 blue, 20 red",20.0,8.0,4.0
5,5,0,"1 red, 3 blue, 15 green",1.0,3.0,15.0
...,...,...,...,...,...,...
569,64,5,"12 green, 4 red\n",4.0,NaN,12.0
573,68,5,"8 green, 8 red\n",8.0,NaN,8.0
574,69,5,"2 green, 4 blue\n",NaN,4.0,2.0
577,72,5,"7 red, 2 blue, 4 green\n",7.0,2.0,4.0


In [58]:
conditions = [
    (df_melt['red'] > 12),
    (df_melt['green'] > 13),
    (df_melt['blue'] > 14),
    (df_melt['red'] <= 12) & (df_melt['green'] <= 13) & (df_melt['blue'] <= 14)
    ]

# create a list of the values we want to assign for each condition
values = [1,1,1,0]

# create a new column and use np.select to assign values to it using our lists as arguments
df_melt['fail'] = np.select(conditions, values)

In [64]:
df_game_results = df_melt.groupby(['gameID']).sum()
df_game_results

<ipython-input-64-eb7a2e17cf47>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_game_results = df_melt.groupby(['gameID']).sum()


,red,blue,green,fail
gameID,,,,
1,9.0,24.0,39.0,1
10,19.0,10.0,39.0,1
100,31.0,34.0,46.0,1
11,4.0,35.0,31.0,2
12,18.0,45.0,57.0,2
...,...,...,...,...
95,11.0,11.0,51.0,1
96,4.0,18.0,12.0,0
97,18.0,2.0,18.0,0


In [76]:
df_success = df_game_results[df_game_results['fail'] == 0]
df_success['gameID2'] = df_success.index
df_success['gameID2'] = pd.to_numeric(df_success['gameID2'])
df_success.sum()

<ipython-input-76-47e5c2c5f63f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_success['gameID2'] = df_success.index
<ipython-input-76-47e5c2c5f63f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_success['gameID2'] = pd.to_numeric(df_success['gameID2'])


red         783.0
blue        675.0
green       710.0
fail          0.0
gameID2    2105.0
dtype: float64

Part 2

In [84]:
df_max = df_melt.groupby(['gameID']).max()
df_max['power'] = df_max['red']*df_max['blue']*df_max['green']


In [86]:
df_max.sum()

variable                                                  349
results      9 blue, 13 green, 1 red 7 green, 5 blue, 7 re...
red                                                    1004.0
blue                                                    846.0
green                                                   926.0
fail                                                       59
power                                                 72422.0
dtype: object